<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Test_CompareImputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install missingpy
!pip install scikit-learn==1.1.2

In [36]:
import sklearn
print(sklearn.__version__)

1.1.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**OriginalData**

In [38]:
import pandas as pd
widetable = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/Wide Table/WideTable.xlsx')
widetable = widetable.iloc[:, 1:]
widetable.head()

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L20:HbA1c,L21:Cholesterol,L22:Triglyceride,L23:LDL Cholesterol,L24:HDL Cholesterol,L28:UrineProtein/Cr ratio,eGFR_t1,time_t1,LabLocation,LabCwhen
0,2299,3386255,-7.158596,90,0,0,0,0,0,0,...,NaN,215.0,81.0,123.0,99.0,NaN,63.60,2017-10-03 09:19:54.710,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-15 09:51:55.203
1,2299,4546471,5.550691,90,90,0,0,0,0,0,...,NaN,219.0,64.0,146.0,73.0,NaN,70.20,2018-12-12 09:18:49.913,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2560-10-03 09:19:54.920
2,2299,5812902,-6.063710,100,100,34,0,0,0,0,...,5.3,193.0,62.0,129.0,65.0,NaN,64.02,2019-12-19 09:51:24.427,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2561-12-12 10:15:36.930
3,2299,6947220,-12.812245,35,0,1,35,0,0,0,...,NaN,205.0,69.0,139.0,76.0,NaN,48.54,2021-03-04 10:01:42.113,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2562-12-19 09:51:24.577
4,2299,8186767,28.287500,100,0,15,0,0,0,0,...,5.5,198.0,74.0,129.0,60.0,NaN,80.47,2022-04-21 08:45:54.580,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2564-03-04 10:07:37.037


In [39]:
df = widetable.copy()

**dummy**

In [40]:
widetable['Sex'] = widetable['Sex'].replace({'ชาย': 0, 'หญิง': 1})

In [41]:
nationality_map = {'ไทย': 0}

caucasian = ['บริติช  (อังกฤษ, สก็อตแลนด์)', 'อเมริกัน', 'เยอรมัน', 'โอมาน', 'ออสเตรเลีย', 'แคนาดา',
             'อิตาลี', 'สวิส', 'ซีเรีย', 'เดนมาร์ก', 'นอร์เวย์', 'ดัตช์', 'สเปน', 'เบลเยียม', 'ไอซ์แลนด์',
             'ฝรั่งเศส', 'เซนต์คิตส์และเนวิส', 'สวีเดน', 'ออสเตรีย', 'ไซปรัส']
asian = ['ลาว','จีน','เวียดนาม','เขมรอพยพ','ไทยลื้อ','ญี่ปุ่น', 'จีน(ไต้หวัน)']
other = ['ไม่ระบุ','อื่นๆ','ไร้สัญชาติ']

for nationality in asian:
    nationality_map[nationality] = 1

for nationality in caucasian:
    nationality_map[nationality] = 2

for nationality in other:
    nationality_map[nationality] = 3

widetable['NATNL'] = widetable['NATNL'].replace(nationality_map)

In [42]:
import numpy as np

In [43]:
widetable['LabLocation'] = np.where(widetable['LabLocation'] == "ห้องปฏิบัติการจุลทรรศน์วินิจฉัย", 1, 0)

In [44]:
df_no_missing = widetable.dropna()
df_no_missing= df_no_missing.reset_index(drop=True)
df_no_missing.shape

(148, 82)

In [45]:
remove = ['Slopes', 'PatientUID', 'PatientVisitUID','InPatient', 'OutPatient', 'Emergency','eGFR_t1','time_t1', 'VisitLocation', 'StartDTTM','EndDttm', 'LabCwhen']
df_no_missing = df_no_missing.drop(columns=remove)
no_missing = df_no_missing.copy()

In [46]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from scipy import stats
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

**สุ่มค่าว่างในข้อมูล**

In [47]:
missing_percentage = widetable.isnull().mean()
missing_probabilities = missing_percentage / missing_percentage.sum()

In [48]:
import numpy as np
df_with_missing = no_missing.copy()
for column in df_with_missing.columns:
    num_missing = int(np.round(missing_percentage[column] * len(df_with_missing)))
    missing_indices = np.random.choice(len(df_with_missing), num_missing, replace=False)
    df_with_missing.loc[missing_indices, column] = np.nan


In [49]:
df_with_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   G1M2:ARB                      148 non-null    float64
 1   G3M1:Statin                   148 non-null    float64
 2   G5M2:NSAIDs                   148 non-null    float64
 3   G1M5:Diuretic                 148 non-null    float64
 4   G1M7:Beta blocker             148 non-null    float64
 5   G1M3:Dihydropyridine CCB      148 non-null    float64
 6   G1M6:MRA                      148 non-null    float64
 7   G1M1:ACEI                     148 non-null    float64
 8   G2M2:Metformin                148 non-null    float64
 9   G2M3:DPP4 inhibitor           148 non-null    float64
 10  G2M1:SU                       148 non-null    float64
 11  G3M3:Fibrate                  148 non-null    float64
 12  G2M5:SGLT2i                   148 non-null    float64
 13  G4M1:

In [ ]:
num_iterations = 100

mse_Mice = []
mape_Mice = []
mae_Mice = []

for _ in range(num_iterations):
    random_missing_indices = {}
    for column in df_with_missing.columns:
        num_missing = int(np.round(missing_probabilities[column] * len(df_with_missing)))
        random_missing_indices[column] = np.random.choice(len(df_with_missing), num_missing, replace=False)

    imputer_mice = IterativeImputer(random_state=0, initial_strategy='median')
    data_mice = df_with_missing.copy()
    for column in df_with_missing.columns:
        data_mice.loc[random_missing_indices[column], column] = np.nan

    data_mice_filled = imputer_mice.fit_transform(data_mice)
    Mice = pd.DataFrame(data_mice_filled, columns=data_mice.columns).round(1)

    # Calculate and append results for MICE
    mse_Mice.append(mean_squared_error(no_missing, Mice))
    true_values_mice = no_missing.values.flatten()
    imputed_values_mice = Mice.values.flatten()

    non_zero_mask_mice = true_values_mice != 0
    true_values_non_zero_mice = true_values_mice[non_zero_mask_mice]
    imputed_values_non_zero_mice = imputed_values_mice[non_zero_mask_mice]

    mape_Mice.append(np.mean(np.abs((true_values_non_zero_mice - imputed_values_non_zero_mice) / true_values_non_zero_mice)) * 100)
    mae_Mice.append(mean_absolute_error(no_missing, Mice))

In [64]:
mape_Mice

[22.845882361191038,
 21.268704200491882,
 19.679010448132246,
 26.656816316872405,
 18.76199713402605,
 22.62535578905487,
 19.825979201061237,
 23.7471001373801,
 22.68228824874288,
 22.23128723561025,
 25.823712207419796,
 20.203179530990965,
 22.202021749136026,
 21.25921807726943,
 24.72058931542461,
 22.385499364246243,
 16.771042663002305,
 24.136077622886962,
 21.7634983797872,
 22.312270300553383,
 12.175546169608142,
 26.51220806856408,
 20.49263166711903,
 17.15242195057584,
 19.21133566636216,
 22.31722607977378,
 20.882883391616115,
 22.373634671815598,
 22.313022865653803,
 20.557623955301153,
 13.3941215761288,
 34.7829712529724,
 18.3873961261489,
 24.72782579328881,
 21.657962709014605,
 13.166389273764228,
 18.710747513447068,
 20.87344177114476,
 19.854821286307082,
 21.11346499722545,
 20.724503892333487,
 17.451099792077628,
 18.201561898452418,
 22.306261819636276,
 23.56368195332444,
 19.107801214724304,
 17.47650686206454,
 23.875222168415053,
 19.63221836446388

In [53]:
mse_Mice

[821.1734587258688,
 636.1546959845559,
 702.70106992278,
 847.9642851737452,
 677.9861554440155,
 599.667453899614,
 584.4820548648648,
 647.011884980695,
 649.4623233976835,
 603.9315083397684,
 639.0853116216216,
 657.5385683783784,
 661.9610274517374,
 598.3879492664094,
 765.2434952123552,
 764.3024483011583,
 559.8482008108108,
 676.981080926641,
 597.2297950193051,
 875.0531963706563,
 583.5405125868725,
 773.3546548648648,
 580.4722013899615,
 542.3739567953668,
 644.9991087258687,
 581.6748208880308,
 678.7725778378377,
 708.7837477220077,
 575.6940303474904,
 581.2376037065637,
 631.2811239768339,
 1001.819048301158,
 656.3989029343629,
 690.148572046332,
 593.931585945946,
 620.0937733976833,
 939.3514768725868,
 720.1083687644789,
 544.2404289961391,
 959.0446085328186,
 615.3927791891892,
 586.7007726254825,
 623.1478455984555,
 567.7149307335907,
 566.5075017760618,
 640.3028905791505,
 541.366078996139,
 700.5552952123552,
 570.6684643243243,
 795.4105100772201,
 656.597

In [51]:
mae_Mice

[3.8659594594594595,
 3.570901544401545,
 3.454996138996139,
 3.939011583011583,
 3.410872586872587,
 3.5012548262548266,
 3.4429305019305017,
 3.6450405405405406,
 3.601658301158301,
 3.4938841698841703,
 3.7467181467181465,
 3.4108822393822393,
 3.650305019305019,
 3.4731891891891897,
 3.8063108108108104,
 3.6459594594594593,
 3.286330115830116,
 3.6620521235521233,
 3.477320463320463,
 3.9834382239382244,
 2.9249961389961388,
 3.8870308880308877,
 3.32142084942085,
 3.146951737451737,
 3.4718397683397684,
 3.4528243243243244,
 3.63084555984556,
 3.6694189189189186,
 3.427934362934363,
 3.4479092664092668,
 3.1349266409266408,
 4.476434362934364,
 3.56834555984556,
 3.673967181467181,
 3.5639073359073357,
 3.0900714285714286,
 3.679277992277992,
 3.6772335907335907,
 3.384083011583012,
 3.897270270270271,
 3.4170984555984556,
 3.3273455598455604,
 3.3015154440154437,
 3.446380308880309,
 3.474028957528957,
 3.42542277992278,
 3.2351602316602324,
 3.6936158301158293,
 3.38966023166023

In [ ]:
num_iterations = 100

mse_MF = []
mape_MF = []
mae_MF = []

for _ in range(num_iterations):
    random_missing_indices = {}
    for column in df_with_missing.columns:
        num_missing = int(np.round(missing_probabilities[column] * len(df_with_missing)))
        random_missing_indices[column] = np.random.choice(len(df_with_missing), num_missing, replace=False)

    imputer_missforest = MissForest(random_state=0, verbose=1)
    data_mf = df_with_missing.copy()
    for column in df_with_missing.columns:
        data_mf.loc[random_missing_indices[column], column] = np.nan

    data_mf_filled = imputer_missforest.fit_transform(data_mf)
    MF = pd.DataFrame(data_mf_filled, columns=data_mf.columns).round(1)

    # Calculate and append results for MF
    mse_MF.append(mean_squared_error(no_missing, MF))
    true_values_mf = no_missing.values.flatten()
    imputed_values_mf = MF.values.flatten()

    non_zero_mask_mf = true_values_mf != 0
    true_values_non_zero_mf = true_values_mf[non_zero_mask_mf]
    imputed_values_non_zero_mf = imputed_values_mf[non_zero_mask_mf]

    mape_MF.append(np.mean(np.abs((true_values_non_zero_mf - imputed_values_non_zero_mf) / true_values_non_zero_mf)) * 100)
    mae_MF.append(mean_absolute_error(no_missing, MF))

In [54]:
mse_MF

[805.4318380694982,
 669.1093450193051,
 697.7445481081081,
 646.4111639382239,
 636.7010585328186,
 706.2243732046331,
 635.782656988417,
 713.870714903475,
 661.9408168339769,
 555.7086606563707,
 674.4733324710423,
 714.7747288030887,
 706.2765531274132,
 729.3853475289575,
 698.5360303474904,
 828.2294006177605,
 687.6089320849419,
 581.7393861389961,
 680.940651969112,
 691.7048048648649,
 658.0673585328186,
 550.3676245559844,
 622.4623587258686,
 638.0665913513512,
 692.4268786100386,
 604.3595071814672,
 612.0981008108107,
 559.2856216602316,
 668.9797822779924,
 675.8417247490347,
 651.3773706949808,
 686.5288550579152,
 546.84031992278,
 807.0717504247104,
 749.8677903861004,
 673.6164838223939,
 690.298282084942,
 615.1442315057914,
 624.6367168339767,
 564.3189372972973,
 632.7985255212355,
 634.6432587258688,
 732.252688069498,
 548.7359689575289,
 725.7179629729729,
 716.3856801544401,
 603.0364513899614,
 618.5876583397683,
 571.3872649034749,
 665.6540594980694,
 704.89

In [55]:
mape_MF

[35.706778987988635,
 31.700494804814618,
 32.142564055627794,
 28.68263821153625,
 31.46996357479624,
 32.316662308536095,
 28.361179232638445,
 31.696915480874555,
 29.896722397957546,
 25.413030300139077,
 31.395606408697933,
 33.62972722025541,
 33.53146283444671,
 32.140876714649046,
 31.908359125074035,
 31.991378999563132,
 30.724485864442624,
 25.498927947404542,
 31.829613591182227,
 27.080087196471197,
 30.330204746454886,
 12.835649220512444,
 25.054743239156373,
 29.868841391052648,
 31.667715622715757,
 27.32662730221681,
 27.076687271100603,
 11.867000692324753,
 31.10540617886831,
 32.71711919171385,
 30.572271518707232,
 32.26145076255494,
 21.702510913024714,
 35.299804014653546,
 35.94996305904781,
 30.6863248287578,
 30.904142524058493,
 28.17630362857384,
 28.950088007847796,
 12.885371278655336,
 28.303444688935976,
 28.59060704346031,
 31.701312580548784,
 12.14865114259671,
 31.05401194186781,
 27.01083864834008,
 28.327939313963974,
 27.797770388247834,
 27.0979

In [56]:
mae_MF

[4.079891891891893,
 3.6904054054054054,
 3.683052123552123,
 3.6181235521235524,
 3.6813030888030887,
 3.7707702702702703,
 3.6227722007722014,
 3.7126563706563704,
 3.5131930501930504,
 3.3323532818532824,
 3.6248745173745176,
 3.7439131274131277,
 3.8074015444015443,
 3.780880308880309,
 3.6413590733590735,
 3.9214111969111967,
 3.6423822393822394,
 3.5656467181467177,
 3.683627413127413,
 3.7274498069498074,
 3.550084942084942,
 2.9340135135135132,
 3.5229054054054054,
 3.6818706563706565,
 3.6678146718146714,
 3.5482741312741313,
 3.5751216216216215,
 2.8954208494208493,
 3.5880057915057915,
 3.8259903474903467,
 3.734507722007722,
 3.688990347490347,
 3.324913127413128,
 4.185598455598456,
 3.8622741312741313,
 3.571561776061776,
 3.739391891891892,
 3.5520444015444013,
 3.6106544401544407,
 2.9235598455598457,
 3.590621621621622,
 3.618233590733591,
 3.779413127413128,
 2.844457528957529,
 3.7781911196911193,
 3.848972972972973,
 3.419444015444016,
 3.6743552123552123,
 3.478739

**SimpleImpute**
```
* Mode = If the data type is 'object'.
* Mean = If the data type is numeric (not 'object') and dont have outlier
* Median = If the data type is numeric (not 'object') and have outlier
```

In [57]:
df = widetable.copy()

In [58]:
# Repeat similar process for Simple Impute
columns_to_keep = ['PatientUID', 'PatientVisitUID', 'Slopes', 'InPatient', 'OutPatient', 'Emergency', 'eGFR_t1','time_t1', 'VisitLocation', 'StartDTTM', 'EndDttm', 'LabCwhen']
df_subset = df.loc[df_with_missing.index]
missing_for_binary = pd.concat([df_subset[columns_to_keep], df_with_missing], axis=1)

for column in df_with_missing.columns:
    missing_for_binary.loc[random_missing_indices[column], column] = np.nan

def detect_outliers(data, column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return not outliers.empty

columns_with_null = missing_for_binary.columns[missing_for_binary.isnull().any()].tolist()
imputed_table = missing_for_binary.copy()

In [59]:
def fill_missing_values(data, column, method):
    if method == 'mean':
        data[column].fillna(data[column].mean(), inplace=True)
    elif method == 'median':
        data[column].fillna(data[column].median(), inplace=True)
    elif method == 'mode':
        data[column].fillna(data[column].mode()[0], inplace=True)
    imputed_table = df_with_missing.copy()

In [60]:
num_iterations = 100

mse_simple = []
mae_simple = []
mape_simple = []

for _ in range(num_iterations):
    # Move the generation of random missing indices inside the loop
    random_missing_indices = {}
    for column in df_with_missing.columns:
        num_missing = int(np.round(missing_probabilities[column] * len(df_with_missing)))
        random_missing_indices[column] = np.random.choice(len(df_with_missing), num_missing, replace=False)

    imputed_table = missing_for_binary.copy()  # Make sure to copy the DataFrame

    for column in columns_with_null:
        if column in imputed_table.select_dtypes(include=['object']).columns:
            # Fill null values with a randomly chosen mode value for object columns
            mode_values = imputed_table[column].mode()
            imputed_table.loc[random_missing_indices[column], column] = np.random.choice(mode_values)
        elif column in imputed_table.select_dtypes(include=['float64']).columns:
            if detect_outliers(df_subset, column):
                # Median imputation for numeric columns with outliers
                median_values = df_subset.groupby('PatientVisitUID')[column].median()
                imputed_table.loc[random_missing_indices[column], column] = imputed_table.loc[random_missing_indices[column], 'PatientVisitUID'].map(median_values)
            else:
                # Mean imputation for numeric columns without outliers
                mean_values = df_subset.groupby('PatientVisitUID')[column].mean()
                imputed_table.loc[random_missing_indices[column], column] = imputed_table.loc[random_missing_indices[column], 'PatientVisitUID'].map(mean_values)

    # Check for outliers in each column and fill missing values accordingly
    for column in columns_with_null:
        outliers = detect_outliers(imputed_table, column)
        if imputed_table[column].dtype == 'object':
            imputed_table[column].fillna(imputed_table[column].mode()[0], inplace=True)
        elif not outliers:
            imputed_table[column].fillna(imputed_table[column].mean(), inplace=True)
        else:
            imputed_table[column].fillna(imputed_table[column].median(), inplace=True)

    # Drop columns if needed
    imputed_table = imputed_table.drop(columns=columns_to_keep)

    # Calculate and append results
    mse_simple.append(mean_squared_error(no_missing, imputed_table))
    true_values_simple = no_missing.values.flatten()
    imputed_values_simple = imputed_table.values.flatten()

    non_zero_mask_simple = true_values_simple != 0
    true_values_non_zero_simple = true_values_simple[non_zero_mask_simple]
    imputed_values_non_zero_simple = imputed_values_simple[non_zero_mask_simple]

    mape_simple.append(np.mean(np.abs((true_values_non_zero_simple - imputed_values_non_zero_simple) / true_values_non_zero_simple)) * 100)
    mae_simple.append(mean_absolute_error(no_missing, imputed_table))


In [65]:
mse_simple

[576.5702850486044,
 514.875022890098,
 518.1309092569329,
 522.4037080401324,
 515.8422140106703,
 506.01287952106236,
 513.0319609793787,
 610.6205511753247,
 517.0536773059446,
 509.54483419760487,
 510.0390977425267,
 516.6249237358803,
 522.2421523380153,
 513.8889398723535,
 521.0335076762163,
 522.3072394466853,
 535.4195919588009,
 512.0790435029107,
 517.3113006102913,
 517.3215016251365,
 608.3956735857682,
 590.8216878060771,
 517.5030450009396,
 512.3339663545205,
 511.5539889370321,
 517.0912406955741,
 525.9293958068478,
 514.5713155196966,
 514.2669840589209,
 518.2159251089048,
 525.5402794157571,
 516.3592471075464,
 510.3533980799021,
 512.2523315177557,
 510.04625204974855,
 513.5205127934647,
 524.3586488031824,
 515.0301599853296,
 518.8521203173245,
 513.1220105320002,
 518.1545708235113,
 588.4297683093009,
 513.6702939353859,
 508.12880003026345,
 518.9498773592039,
 514.6195701973968,
 516.0763100342855,
 513.0136560969238,
 583.6481321134875,
 518.233521126404

In [66]:
mae_simple

[3.0786890933127937,
 3.0360175353321694,
 3.055454504546815,
 3.064341525487425,
 3.037743136554982,
 3.0010609100264225,
 3.06502319282415,
 3.1219684313719305,
 3.045345588709034,
 3.0404922405004404,
 3.00928035933531,
 3.0426265879681433,
 3.065357654166985,
 3.0349279691799396,
 3.0428191385934107,
 3.0599447143027705,
 3.0029350585482484,
 3.0058425787061487,
 3.0164746987453848,
 3.042725975604658,
 3.1684324428376045,
 3.1555432183335403,
 3.0492869588296863,
 3.0433456099417184,
 2.997091011429721,
 3.0292326472030773,
 3.088142156442156,
 3.05054660884428,
 3.0444964289326104,
 3.01680173202932,
 3.09682622655419,
 3.0457030600736648,
 3.01465061819164,
 3.0013613153904637,
 2.999824659421139,
 3.027325288893983,
 3.0630076702758378,
 3.015733123726677,
 3.06585955729741,
 3.0549436676834127,
 3.0669061903634844,
 3.100032221109059,
 3.0179407464932613,
 3.000384847647181,
 3.066841413683851,
 3.0403757035637846,
 3.032064411362782,
 3.0392762428754114,
 3.125806429600598,
 

In [61]:
mape_simple

[15.193981174806066,
 13.586574180971573,
 13.744405303257611,
 13.566480144545148,
 13.677213152866047,
 13.407706098759745,
 13.882558917433812,
 11.678276511991031,
 13.5702169849721,
 13.719397841690654,
 13.425140361875718,
 13.685939449185359,
 13.768203097010504,
 13.354282835793848,
 13.199203022358564,
 13.417570455897035,
 11.524207787230434,
 13.471169592157286,
 13.217105129805082,
 13.581845138955087,
 12.626338876724699,
 13.398024486021837,
 13.188028919504536,
 13.623713285190822,
 13.349069160322893,
 13.468661423078416,
 13.851363579488568,
 13.826256872492944,
 13.586619919570845,
 13.147902578088683,
 13.428013045390383,
 13.788291374785528,
 13.390848713683628,
 13.31798048939154,
 13.618773919181873,
 13.515157076364634,
 13.465234212937983,
 13.14900462651098,
 13.566685931839132,
 13.811164553529931,
 13.743591494441104,
 12.463087935164587,
 13.312186631779452,
 13.689199026453569,
 15.217377753804085,
 13.760847327569767,
 13.202554949944279,
 13.6406789728170

#**Compare**



In [62]:
import pandas as pd
import numpy as np

# Assuming you have the following results:
mae_results = {'MF': mae_MF, 'Mice': mae_Mice, 'Simple': mae_simple}
mse_results = {'MF': mse_MF, 'Mice': mse_Mice, 'Simple': mse_simple}
mape_results = {'MF': mape_MF, 'Mice': mape_Mice, 'Simple': mape_simple}

# Calculate means of the results
mean_mse_results = {method: np.mean(results) for method, results in mse_results.items()}
mean_mape_results = {method: np.mean(results) for method, results in mape_results.items()}
mean_mae_results = {method: np.mean(results) for method, results in mae_results.items()}

# Convert means to a DataFrame for easier printing
mean_evaluation_df = pd.DataFrame({
    'Method': list(mean_mse_results.keys()),
    'Mean MSE': list(mean_mse_results.values()),
    'Mean MAPE': list(mean_mape_results.values()),
    'Mean MAE': list(mean_mae_results.values())
})

mean_evaluation_df


,Method,Mean MSE,Mean MAPE,Mean MAE
0,MF,653.726426,28.351770,3.582946
1,Mice,672.018913,21.446576,3.565713
2,Simple,522.564686,13.587867,3.045545


In [63]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

imputed_data = {'OriginalData': no_missing}
imputed_data['Mice'] = Mice
imputed_data['MissForest'] = MissForest
imputed_data['SimpleImpute'] = imputed_table